In [1]:
#1. 載入相關模組
from keras.optimizers import Adam
from keras.applications import ResNet50
from keras import models
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import optimizers


# 以訓練好的 ResNet50 為基礎來建立模型，
# 捨棄 ResNet50 頂層的 fully connected layers

cnn_base = ResNet50(include_top=False, weights='imagenet',input_shape=(224, 224, 3))


classify = models.Sequential()
classify.add(layers.Flatten())
classify.add(layers.Dense(512, activation='relu'))
classify.add(layers.Dropout(0.5))
classify.add(layers.Dense(5, activation='sigmoid'))
#classify.summary()

#5. 將CNN base與自定的分類器組合起來
model = models.Sequential()
model.add(cnn_base)
model.add(classify)

model.summary()


Using TensorFlow backend.
C:\Users\Richard\anaconda3\envs\tensorflowgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Richard\anaconda3\envs\tensorflowgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Richard\anaconda3\envs\tensorflowgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Richard\

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
sequential_1 (Sequential)    (None, 5)                 51383301  
Total params: 74,971,013
Trainable params: 74,917,893
Non-trainable params: 53,120
_________________________________________________________________


In [2]:


train_batches = ImageDataGenerator(
        rotation_range = 30,        #rotation_range：整數，資料提升時圖片隨機轉動的角度。隨機選擇圖片的角度，是一個0~180的度數，取值為0~180。
        width_shift_range = 0.3,   #width_shift_range：浮點數，圖片寬度的某個比例，資料提升時圖片隨機水平偏移的幅度
        height_shift_range = 0.3,  #height_shift_range：浮點數，圖片高度的某個比例，資料提升時圖片隨機豎直偏移的幅度。
        shear_range = 0.3,
        zoom_range = 0.4,
        channel_shift_range = 0.2,
        fill_mode = 'nearest',
        horizontal_flip = True,
        vertical_flip = False).flow_from_directory(
    'training', target_size=(224,224), class_mode='categorical', batch_size=10)

valid_batches = ImageDataGenerator(
        rotation_range = 30,
        width_shift_range = 0.3,
        height_shift_range = 0.3,
        shear_range = 0.3,
        zoom_range = 0.4,
        channel_shift_range = 0.2,
        fill_mode = 'nearest',
        horizontal_flip = True,
        vertical_flip = False).flow_from_directory(
    'testing', target_size=(224,224), class_mode='categorical', batch_size=10)



Found 2322 images belonging to 5 classes.
Found 481 images belonging to 5 classes.


In [3]:

# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
model.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])


# 訓練模型
#net_final.fit_generator(train_batches,
                       # steps_per_epoch =50,
                       # validation_data = valid_batches,
                       # validation_steps = 50,
                       # epochs = 10, verbose=2)

model.fit_generator(train_batches, steps_per_epoch=50,epochs=100,validation_data=valid_batches, validation_steps=50, verbose=2)

Epoch 1/100
 - 29s - loss: 1.6657 - acc: 0.3060 - val_loss: 1.5682 - val_acc: 0.2648
Epoch 2/100
 - 15s - loss: 1.4688 - acc: 0.4720 - val_loss: 1.5885 - val_acc: 0.4012
Epoch 3/100
 - 15s - loss: 1.2058 - acc: 0.5840 - val_loss: 1.5391 - val_acc: 0.4705
Epoch 4/100
 - 15s - loss: 1.0304 - acc: 0.6360 - val_loss: 1.6514 - val_acc: 0.4868
Epoch 5/100
 - 16s - loss: 1.0988 - acc: 0.6420 - val_loss: 1.3334 - val_acc: 0.5214
Epoch 6/100
 - 15s - loss: 0.8841 - acc: 0.7480 - val_loss: 1.3976 - val_acc: 0.5275
Epoch 7/100
 - 15s - loss: 0.7946 - acc: 0.7380 - val_loss: 1.3504 - val_acc: 0.5703
Epoch 8/100
 - 15s - loss: 0.7743 - acc: 0.7480 - val_loss: 1.3331 - val_acc: 0.5927
Epoch 9/100
 - 15s - loss: 0.7553 - acc: 0.7400 - val_loss: 1.1930 - val_acc: 0.6049
Epoch 10/100
 - 15s - loss: 0.7254 - acc: 0.7601 - val_loss: 1.1223 - val_acc: 0.5927
Epoch 11/100
 - 15s - loss: 0.6016 - acc: 0.8060 - val_loss: 1.4511 - val_acc: 0.5601
Epoch 12/100
 - 15s - loss: 0.6395 - acc: 0.7960 - val_loss: 1.

In [4]:
#同時儲存結構與權重，檔案的類別為HDF5
from keras.models import load_model
model.save('flower6.h5')
print(train_batches.class_indices.items())

dict_items([('daisy', 0), ('dandelion', 1), ('rose', 2), ('sunflower', 3), ('tulip', 4)])


In [5]:
#載入HDF5檔案
from keras.models import load_model
# 載入模型
classifier = load_model('flower6.h5')

In [7]:
import sys,os,dlib,glob
#from skimage import io
from keras.preprocessing import image
import numpy as np

folder_path='exam2\\'
# 列出指定路徑底下所有檔案(包含資料夾)
allFileList = os.listdir(folder_path)
# 逐一查詢檔案清單

listlable = []  
for file in allFileList:
    f= folder_path+ file
    file=os.path.splitext(file)[0]
    #print(f)
    
    img = image.load_img(f, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
   
    res=classifier.predict_classes(x) 
    print(file,res)


0 (1) [0]
0 (2) [0]
0 (3) [0]
1 (1) [0]
1 (2) [0]
1 (3) [0]
2 (1) [0]
2 (2) [0]
2 (3) [0]
3 (1) [0]
3 (2) [0]
3 (3) [0]
4 (1) [0]
4 (2) [0]
4 (3) [0]
